In [4]:
import dash
from dash import dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import json

In [5]:
df = pd.read_csv('HIST_PAINEL_COVIDBR_2022_Parte1_10jun2022.csv', sep=';', on_bad_lines='skip')

#### First Step: Filtering the data

In [6]:
#df_states = df[~df['estado'].isna()]
#df_brazil = df[df['regiao'] == 'Brasil']

In [8]:
#df_states.to_csv('df_states.csv')
#df_brazil.to_csv('df_brazil.csv')

In [37]:
df_states = pd.read_csv('df_states.csv')
df_brazil = pd.read_csv('df_brazil.csv')

In [19]:
brazil_states = json.load(open('geojson/brazil_geo.json', 'r'))
type(brazil_states)
brazil_states.keys()
type(brazil_states['features'])
type(brazil_states['features'][0])

dict

In [18]:
brazil_states['features'][0].keys()

dict_keys(['type', 'id', 'properties', 'geometry'])

In [17]:
brazil_states['features'][0]['geometry']

{'type': 'Polygon',
 'coordinates': [[[-73.3325109999999, -7.324878999999896],
   [-73.274817, -7.350334],
   [-72.870164, -7.528881999999896],
   [-72.691514, -7.610621],
   [-72.659348, -7.624979],
   [-72.189666, -7.720489],
   [-72.173179, -7.723841],
   [-72.023178, -7.769827],
   [-72.005143, -7.774846],
   [-71.569238, -7.874255],
   [-71.5129859999999, -7.887299],
   [-71.241607, -7.950207],
   [-70.937416, -8.01909],
   [-70.575844, -8.09594],
   [-70.55023099999988, -8.101379999999892],
   [-70.462364, -8.120046],
   [-70.427306, -8.12811],
   [-70.369521, -8.141402],
   [-70.296363, -8.180022],
   [-70.291695, -8.182485],
   [-70.283623, -8.186811],
   [-70.12950099999989, -8.269364],
   [-69.7612769999999, -8.466229],
   [-69.641479, -8.530838],
   [-69.553328, -8.57838],
   [-69.435775, -8.641597],
   [-69.178231, -8.767959],
   [-69.162169, -8.776353],
   [-68.989593, -8.866563],
   [-68.784995, -8.972589],
   [-68.750061, -8.988709],
   [-68.734749, -8.996609],
   [-68.7

In [22]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])

In [58]:
df_states.head()

,Unnamed: 0,regiao,estado,municipio,coduf,codmun,codRegiaoSaude,nomeRegiaoSaude,data,semanaEpi,populacaoTCU2019,casosAcumulado,casosNovos,obitosAcumulado,obitosNovos,Recuperadosnovos,emAcompanhamentoNovos,interior/metropolitana
0,161,Norte,RO,NaN,11,NaN,NaN,NaN,2022-01-01,52,1777225.0,284660.0,34,6737,3,NaN,NaN,NaN
1,162,Norte,RO,NaN,11,NaN,NaN,NaN,2022-01-02,1,1777225.0,284692.0,32,6739,2,NaN,NaN,NaN
2,163,Norte,RO,NaN,11,NaN,NaN,NaN,2022-01-03,1,1777225.0,284783.0,91,6742,3,NaN,NaN,NaN
3,164,Norte,RO,NaN,11,NaN,NaN,NaN,2022-01-04,1,1777225.0,285037.0,254,6745,3,NaN,NaN,NaN
4,165,Norte,RO,NaN,11,NaN,NaN,NaN,2022-01-05,1,1777225.0,285269.0,232,6748,3,NaN,NaN,NaN


In [56]:
df_states_ = df_states[df_states['data'] == '2022-01-10']

In [57]:
fig = px.choropleth_mapbox(df_states_, locations='estado', color='casosNovos', geojson='brazil_states',
                           center= {'lat': -16.95, 'lon': -47.78},
                           zoom=4,
                           color_continuous_scale='Redor', opacity=0.4, 
                           hover_data={'casosAcumulado':True, 'casosNovos':True, 'obitosNovos':True, 'estado':True})

fig.update_layout(
    paper_bgcolor='#242424',
    autosize=True,
    margin=None,
    showlegend=False,
    mapbox_style='carto-darkmatter'
)

In [46]:
app.layout = dbc.Container(
    dbc.Row([
        dbc.Col([
            dcc.Graph(id='choropleth-map', figure=fig)
        ])
    ])
)

if __name__ == "__main__":
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

